<a href="https://colab.research.google.com/github/NilmaraSalgado/EducacaoBemolDigital/blob/cursoEstatistica/Aula_Estat%C3%ADstica_Bemol_Educa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Aula 5: Problema Prático: Utilizando resultados de Teste A/B**

### 1 Definir o objetivo da análise

**Queremos avaliar a satisfação de nossos clientes em relação ao produto X recém lançado.**

- NPS - Net Promoter Score
- Teste A/B

   Dois tipos de mensagens foram enviadas. 
  - **Texto A:** *BEMOL: Olá, NOME! você utilizou o produto X. Avalie sua experiência com o serviço de 0 a 10. Resposta gratuita. Obrigado!*

  - **Texto B:** *BEMOL: Olá, você utilizou o produto X. Avalie sua experiência com o serviço de 0 a 10. Resposta gratuita.*

### 2 Coletar os dados

* **ID_SMS.csv** 
  - IDS_SMS (coluna) str : identificador único do SMS
  - tp (coluna) str : tipo do texto SMS A ou B

* **RESPOSTAS.xlsx**
  - ID_SMS (coluna) str : identificador único do SMS
  - RESPONSE (coluna) str : resposta dos clientes

### 3 Carregar os dados no ambiente de análise e remover os campos desnecessários

In [ ]:
import pandas as pd

In [ ]:
# apesar de chamarmos frame de dados, considere como se você estivesse abrindo
# uma planilha no Excel
df_ids = pd.read_csv('./IDS_SMS.csv')
df_respostas = pd.read_excel('./RESPOSTAS.xlsx')

In [ ]:
# identificando o quantidade de linhas e colunas dos dados
df_ids.shape

In [ ]:
df_respostas.shape

In [ ]:
# prévia da base
df_ids.head()

In [ ]:
df_respostas.head()

In [ ]:
# verificando quantos valores nulos temos por colunas.
df_ids.isna().sum()

In [ ]:
df_respostas.isna().sum()

In [ ]:
# mostrando os valores nulos da tabela de resposta
df_respostas[df_respostas['RESPONSE'].isnull()]

In [ ]:
# removendo os valores nulos da tabela de resposta
df_respostas.dropna(inplace=True)

In [ ]:
# verificando novamente valores nulos após remover
df_respostas.isna().sum()

In [ ]:
# frequencia de valores
df_ids['tp'].value_counts()

In [ ]:
df_respostas['RESPONSE'].value_counts().head(30)

In [ ]:
# interessante, tem números escrito por extenso
filtro = df_respostas['RESPONSE'].str.lower().isin(['dez','oito'])

In [ ]:
# já é um indicativo que podemos tentar ajustar algumas notas.
df_respostas[filtro]

In [ ]:
# vamos tentar identificar notas dentro de respostas com texto
# a. ou vc remove
# b. ou vc trata

In [ ]:
#importar módulo para trabalhar com expressões regulares
import re

In [ ]:
#função para extrair nota do texto
def extrair_nota(texto):
  '''
    Extrai o número do texto
  '''
  try:
    return re.findall(r'\d+', str(texto))[0]
  except:
    return texto

In [ ]:
# se ficou curioso olha uns testes aqui da função
extrair_nota('nossa, eu gostei muito nota 10')

In [ ]:
extrair_nota('muito bom, adorei')

In [ ]:
extrair_nota('gente, nota 100000 vocês são d+')

In [ ]:
df_respostas.head(10)

In [ ]:
# criando uma coluna nova e armazenando as notas extraídas
df_respostas['NOTA_EXTRAIDA'] = df_respostas['RESPONSE'].apply(extrair_nota)

In [ ]:
df_respostas['NOTA_EXTRAIDA'].value_counts()

In [ ]:
# antes de tirar as notas que não são palavras, vamos tentar transformar 
# os texto de números escrito por extenso

filtro_npe = df_respostas['NOTA_EXTRAIDA'].str.lower().isin(['oito','dez'])
filtro_n = df_respostas['NOTA_EXTRAIDA'].str.isnumeric()

In [ ]:
# vamos rescrever nossa planilha com base nos filtros
df_respostas = df_respostas[filtro_npe | filtro_n].copy()

In [ ]:
df_respostas['NOTA_EXTRAIDA'].value_counts()

In [ ]:
#função para retornar valor númerico
def conv_extenso_num(nota):
  if nota.lower() == 'dez':
    return 10
  elif nota.lower() == 'oito':
    return 8
  else:
    return nota

In [ ]:
df_respostas['NOTA_EXTRAIDA2'] = df_respostas['NOTA_EXTRAIDA'].apply(conv_extenso_num)

In [ ]:
df_respostas.head()

In [ ]:
# agora sim temos somente numeros em nossa coluna notas
df_respostas['NOTA_EXTRAIDA2'].value_counts()

In [ ]:
df_respostas['NOTA_EXTRAIDA2'] = df_respostas['NOTA_EXTRAIDA2'].astype(int)

In [ ]:
df_respostas['NOTA_EXTRAIDA2'].value_counts()

In [ ]:
# podemos converter os super notas em notas 10
f = lambda nota: 10 if nota > 10 else nota
df_respostas['NOTA_EXTRAIDA2'] = df_respostas['NOTA_EXTRAIDA2'].apply(f)

In [ ]:
df_respostas['NOTA_EXTRAIDA2'].value_counts()

In [ ]:
# calculando as frequencias
def calc_freq_a(notas):
  return len(notas)

def calc_freq_r(notas):
  return len(notas) / df_respostas.shape[0] * 100

In [ ]:
calcs = {'ID_SMS':[calc_freq_a, calc_freq_r]}

df_respostas.groupby(by=['NOTA_EXTRAIDA2'], as_index=False).\
             agg(calcs).sort_values('NOTA_EXTRAIDA2', ascending=False)

In [ ]:
# vez que fosse solicitado.
df_respostas = df_respostas[['ID_SMS','NOTA_EXTRAIDA2']].rename(columns={'NOTA_EXTRAIDA2':'NOTA'})

In [ ]:
df_ids.head()

In [ ]:
df_respostas.head()

In [ ]:
# identificando o tipo de texto da nota
df_notas = df_respostas.merge(df_ids)

In [ ]:
df_notas.shape

In [ ]:
df_notas.isna().sum()

In [ ]:
df_notas.sample(10).head(10)

In [ ]:
df_notas.columns = ['ID','NOTA','TIPO']

In [ ]:
df_notas.sample(10).head(10)

### 4 Começar analisar os dados

In [ ]:
dic_agg = {'NOTA':['count','min','max','std','mean']}

In [ ]:
resumo = df_notas.groupby(by=['TIPO'], as_index=False).agg(dic_agg)
resumo

In [ ]:
df_ids['tp'].value_counts()

In [ ]:
# vamos criar um coluna para armazenar a quantidade de sms enviado
resumo['sms_enviados'] = 1500
resumo

In [ ]:
cols = ['TIPO'] + list(filter(None, resumo.columns.droplevel()))\
                + ['sms_enviados']

resumo.columns = cols

In [ ]:
resumo

In [ ]:
resumo['engajamento'] = resumo['count'] / resumo['sms_enviados']

In [ ]:
resumo

**NPS - Net Promoter Score**

1. Promotores (nota 9 ou 10)
2. Neutros (nota 7 ou 8)
3. Detratores (nota de 0 a 6)

Como cacular?
* *% CLIENTES PROMOTORES – % CLIENTES DETRATORES = NPS*


In [ ]:
# criando uma função para calcular a classe de NPS
def classe_nps(nota):
  if (nota >= 9):
    return 'PROMOTORES'
  elif (nota >= 7) & (nota <= 8):
    return 'NEUTROS'
  else:
    return 'DETRATORES' 

In [ ]:
# para as notas de NPS não se 
df_notas_a = df_notas[df_notas['TIPO'] == 'Texto A'].copy()
df_notas_b = df_notas[df_notas['TIPO'] == 'Texto B'].copy()

In [ ]:
df_notas_a = df_notas_a.groupby(by='NOTA', as_index=False).agg({'TIPO':'count'})
df_notas_b = df_notas_b.groupby(by='NOTA', as_index=False).agg({'TIPO':'count'})

In [ ]:
df_notas_a['classe'] = df_notas_a['NOTA'].apply(classe_nps)
df_notas_b['classe'] = df_notas_b['NOTA'].apply(classe_nps)

In [ ]:
df_notas_a

In [ ]:
df_notas_b

In [ ]:
nps_a = df_notas_a.groupby(by='classe', as_index=False).agg({'TIPO':'sum'})
nps_b = df_notas_b.groupby(by='classe', as_index=False).agg({'TIPO':'sum'})

In [ ]:
nps_a

In [ ]:
nps_b

In [ ]:
# frequencia relativa das classes
nps_a['freq_rel_classe'] = nps_a['TIPO'] / nps_a['TIPO'].sum()
nps_b['freq_rel_classe'] = nps_b['TIPO'] / nps_b['TIPO'].sum()

In [ ]:
nps_b

### 5 Interpretar/apresentar os resultados

In [ ]:
nps_a

In [ ]:
nps_final_a = round(0.983425 - 0.008287,2)
print(f'NPS A: {nps_final_a * 100}')

In [ ]:
nps_b

In [ ]:
nps_final_b = round(0.966667 - 0.007576,2)
print(f'NPS B: {nps_final_b * 100}')